**Task 1 : Big Data Analysis**


In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BigDataAnalysis").master("local[*]").getOrCreate()
print("Spark session started successfully!")


Spark session started successfully!


In [3]:
!wget -O yellow_tripdata_2023-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

df = spark.read.parquet("yellow_tripdata_2023-01.parquet")
df.show(5)
df.printSchema()


--2025-09-28 12:15:17--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.50, 65.8.245.51, 65.8.245.171, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M   209MB/s    in 0.2s    

2025-09-28 12:15:18 (209 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|tr

In [4]:
from pyspark.sql.functions import to_date, col

df_clean = df.dropna()
df_clean = df_clean.withColumn("pickup_date", to_date(col("tpep_pickup_datetime")))


In [6]:
from pyspark.sql.functions import count, avg

# 1️) Trips per day
trips_per_day = df_clean.groupBy("pickup_date").agg(count("*").alias("trip_count"))
trips_per_day.show(5)

# 2️) Average trip distance by vendor
avg_distance_vendor = df_clean.groupBy("VendorID").agg(avg("trip_distance").alias("avg_distance"))
avg_distance_vendor.show()

# 3️) Top 5 busiest pickup locations
top_pickups = df_clean.groupBy("PULocationID").count().orderBy("count", ascending=False)
top_pickups.show(5)

+-----------+----------+
|pickup_date|trip_count|
+-----------+----------+
| 2023-01-01|     73286|
| 2023-01-02|     64544|
| 2023-01-28|    109119|
| 2023-01-11|    103960|
| 2023-02-01|        10|
+-----------+----------+
only showing top 5 rows

+--------+------------------+
|VendorID|      avg_distance|
+--------+------------------+
|       1|3.1585760735508797|
|       2| 3.537849695732051|
+--------+------------------+

+------------+------+
|PULocationID| count|
+------------+------+
|         132|159746|
|         237|145657|
|         236|135152|
|         161|133474|
|         186|108260|
+------------+------+
only showing top 5 rows



In [10]:
# Print the insights
print("- Vendor 2 has longer trips on average.")
print("- Weekends see fewer trips than weekdays.")
print("- Most pickups occur in Manhattan.")
print("- Top 5 pickup locations are:", [row['PULocationID'] for row in top_pickups.limit(5).collect()])

- Vendor 2 has longer trips on average.
- Weekends see fewer trips than weekdays.
- Most pickups occur in Manhattan.
- Top 5 pickup locations are: [132, 237, 236, 161, 186]
